Решение сложных задач с SQL-EX

Какое максимальное количество черных квадратов можно было бы окрасить в белый цвет
оставшейся краской

In [ ]:
select iif(count(*)=3,min(ty),0) as qty from (select COALESCE(sum(remainpaint)/255, 0) as ty
from(select V_COLOR, V_ID,
CASE
WHEN SUM(B_VOL) IS NULL
THEN 255
ELSE 255-SUM(B_VOL)
END RemainPaint
from utB right join utV on B_V_ID = V_ID
group by V_COLOR, V_ID
) R
group by V_COLOR) as kak


По таблицам Income и Outcome для каждого пункта приема найти остатки денежных средств на конец каждого дня,
в который выполнялись операции по приходу и/или расходу на данном пункте.
Учесть при этом, что деньги не изымаются, а остатки/задолженность переходят на следующий день.
Вывод: пункт приема, день в формате "dd/mm/yyyy", остатки/задолженность на конец этого дня.

In [ ]:
with asa as (select coalesce(income.point, outcome.point) as point, coalesce(income.date, outcome.date) as date,
coalesce(sum(income.inc), 0) - coalesce(sum(outcome.out), 0) as balance
from income full join outcome on outcome.date=income.date and outcome.code=income.code and outcome.point=income.point
group by coalesce(income.point, outcome.point), coalesce(income.date, outcome.date))
select point, convert(varchar, date, 103) day, sum(balance) over(partition by point order by date RANGE UNBOUNDED PRECEDING)
from asa

Пронумеровать уникальные пары {maker, type} из Product, упорядочив их следующим образом:
- имя производителя (maker) по возрастанию;
- тип продукта (type) в порядке PC, Laptop, Printer.
Если некий производитель выпускает несколько типов продукции, то выводить его имя только в первой строке;
остальные строки для ЭТОГО производителя должны содержать пустую строку символов ('').

In [ ]:
select row_number() over(order by maker) as num, CASE WHEN mnum=1 THEN maker
    ELSE ''
  END as maker, type
from
( select
    row_number() over(partition by maker order by maker, ord) as mnum
    ,maker
    ,type
    from (
    select
      distinct maker, type
      ,CASE WHEN type='pc' then 1
            WHEN type='laptop' then 2
            ELSE 3
      END as ord
      from product) as mto
) as makaka


При условии, что баллончики с красной краской использовались более одного раза, выбрать из них такие, которыми окрашены квадраты, имеющие голубую компоненту.
Вывести название баллончика

In [ ]:
select distinct kakom.v_name
from
(select b_q_id, odin.v_name, v_color
from (select b_q_id, v_id, v_name, v_color
from utb inner join utv on utb.b_v_id=utv.v_id) as odin inner join (select v_id
from utB inner join utv on utv.v_id=utb.b_v_id
where v_coloR = 'B') as dva on odin.v_id=dva.v_id) as kak inner join
(select b_q_id, odin.v_name
from (select b_q_id, v_id, v_name, v_color
from utb inner join utv on utb.b_v_id=utv.v_id) as odin inner join (select v_id, count(b_q_id) as howoften
from utB inner join utv on utv.v_id=utb.b_v_id
where v_coloR = 'R'
group by v_name, v_id
having count(v_id) > 1) as dva on odin.v_id=dva.v_id) as kakom on kak.b_q_id=kakom.b_q_id


Среди тех, кто пользуется услугами только одной компании, определить имена разных пассажиров, летавших чаще других.
Вывести: имя пассажира, число полетов и название компании.


In [ ]:
SELECT
 (SELECT name FROM Passenger WHERE ID_psg = B.ID_psg) AS name,
 B.trip_Qty,
 (SELECT name FROM Company WHERE ID_comp = B.ID_comp) AS Company
FROM (SELECT P.ID_psg, MIN(T.ID_comp) AS ID_comp, COUNT(*) AS trip_Qty, MAX(COUNT(*)) OVER() AS Max_Qty
      FROM Pass_in_trip AS P JOIN
       Trip AS T ON P.trip_no = T.trip_no
      GROUP BY P.ID_psg
      HAVING MIN(T.ID_comp) = MAX(T.ID_comp)
      ) AS B
WHERE B.trip_Qty = B.Max_Qty


Для каждой компании подсчитать количество перевезенных пассажиров (если они были в этом месяце) по декадам апреля 2003. При этом учитывать только дату вылета.
Вывод: название компании, количество пассажиров за каждую декаду

In [ ]:
WITH AKA AS (select name, CASE
 WHEN date between'2003-04-01 00:00:00.000' and '2003-04-10 00:00:00.000'
 THEN count(date)
 ELSE 0
 END ODIN,
case
 WHEN date between'2003-04-11 00:00:00.000' and '2003-04-20 00:00:00.000'
 THEN count(date)
 ELSE 0
 END DVA,
case
WHEN date between'2003-04-21 00:00:00.000' and '2003-04-30 00:00:00.000'
 THEN count(date)
 ELSE 0
 END TRI
from pass_in_trip inner join trip on trip.trip_no=pass_in_trip.trip_no
inner join company on company.id_comp=trip.id_comp
where date between'2003-04-01 00:00:00.000' and '2003-04-30 00:00:00.000'
group by name, date)
select name, sum(ODIN) as '1-10', sum(dva) as '11-20', sum(tri) as '21-30'
from AKA
group by name

Выбрать три наименьших и три наибольших номера рейса. Вывести их в шести столбцах одной строки, расположив в порядке от наименьшего к наибольшему.
Замечание: считать, что таблица Trip содержит не менее шести строк.


In [ ]:
select min(trip_no) as min1,(select TOP 1 WITH TIES trip_no
from trip
where trip_no not in (select min(trip_no) from trip)
order by trip_no asc) as min2, (select TOP 1 WITH TIES trip_no
from trip
where trip_no not in (select top 2 with ties trip_no from trip order by trip_no asc)
order by trip_no asc) as min3,
(select TOP 1 WITH TIES trip_no
from trip
where trip_no not in (select top 2 with ties trip_no from trip order by trip_no desc)
order by trip_no desc) as max3,
(select TOP 1 WITH TIES trip_no
from trip
where trip_no not in (select max(trip_no) from trip)
order by trip_no desc) as max2,
max(trip_no) as max1
from trip


Сгруппировать все окраски по дням, месяцам и годам. Идентификатор каждой группы должен иметь вид "yyyy" для года, "yyyy-mm" для месяца и "yyyy-mm-dd" для дня.
Вывести только те группы, в которых количество различных моментов времени (b_datetime), когда выполнялась окраска, более 10.
Вывод: идентификатор группы, суммарное количество потраченной краски.

In [ ]:
select CONVERT (varchar, year(b_Datetime)) as year, sum(b_vol) as vol
from utb
group by year(b_Datetime)
having count(distinct B_DATETIME)> 10
union
select rabotai as period, sum(b_vol) as vol
from(
select case
when month(b_datetime)>9 then concat(year(b_datetime), '-', month(b_datetime))
else concat(year(b_datetime), '-','0', month(b_datetime)) end rabotai, b_vol, b_datetime
from utb) as kik
group by rabotai
having count(distinct B_DATETIME)> 10
union
select concat(rabotai, '-', sobak), sum(b_vol)
from (
select case
when month(b_datetime)>9 then concat(year(b_datetime), '-', month(b_datetime))
else concat(year(b_datetime), '-','0', month(b_datetime)) end rabotai, case
when cast(day(b_datetime) as varchar(20))>9 then cast(day(b_datetime) as varchar(20))
else concat('0', cast(day(b_datetime) as varchar(20))) end sobak, b_vol, b_datetime
from utb) as kikim
group by concat(rabotai, '-', sobak)
having count(distinct B_DATETIME)> 10

Считая, что первый пункт вылета является местом жительства, найти пассажиров, которые находятся вне дома. Вывод: имя пассажира, город проживания


In [ ]:
with Namepeople as (select id_psg, town_from, town_to, rOW_NUMBER() OVER(partition by id_psg ORDER BY date+time_out asc) num
from pass_in_trip inner join trip on trip.trip_no=pass_in_trip.trip_no)
select name, town_From
from
(select id_psg, town_from, town_to, num
from (select min(num) as otkuda
from namepeople) as kakakkak inner join namepeople on kakakkak.otkuda=namepeople.num) as mintable
inner join (select kakmaks.id_psg, town_to
from namepeople inner join
(select id_psg, max(num) as kyda
from namepeople
group by id_psg) as kakmaks on kakmaks.kyda=namepeople.num and kakmaks.id_psg=namepeople.id_psg) as kak on kak.id_psg=mintable.id_psg
inner join passenger on passenger.id_psg=kak.id_psg
where mintable.town_from != kak.town_to

Определить имена разных пассажиров, летевших хотя бы два раза подряд на месте с одним и тем же номером.

In [ ]:
with akak as (select id_psg, place, row_number () over(partition by id_psg order by date+time_out) as num, (date+time_out) as alltime
from pass_in_trip inner join trip on trip.trip_no=pass_in_trip.trip_no)
select name
from akak as akodin inner join akak as akdva on akodin.id_psg=akdva.id_psg and akodin.place=akdva.place and akodin.num+1= akdva.num
inner join passenger on passenger.id_psg=akodin.id_psg
group by name, akodin.id_psg


Для каждой пятой модели (в порядке возрастания номеров
моделей) из таблицы Product
определить тип продукции и среднюю цену модели.

In [ ]:
with asa as (select model,type, num, num%5 as number
from (select  model, type, row_number () over(order by model) num
from product) as kakaka
where num % 5=0)
select type, avg_price
from(
select model, type, avg(price) as avg_price
from
(select asa.model, type, price
from asa left join pc on pc.model=asa.model
union all
select asa.model, asa.type, price
from asa left join printer on printer.model=asa.model
union all
select asa.model, asa.type, price
from asa left join laptop on laptop.model=asa.model) as kpi
group by model, type) as kikido


Среди пассажиров, летавших на самолетах только одного типа, определить тех, кто прилетал в один и тот же город не менее 2-х раз.
Вывести имена пассажиров.


In [ ]:
select name
from (select id_psg
from trip inner join pass_in_trip on trip.trip_no=pass_in_trip.trip_no
group by id_psg
having count(distinct plane) = 1
INTERSECT
select id_psg
from trip inner join pass_in_trip on trip.trip_no=pass_in_trip.trip_no
group by town_to, id_psg
having count(town_to) >=2) as anime
inner join passenger on passenger.id_psg=anime.id_psg
